# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import pprint as pprint
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ciudad bolivar,8.1222,-63.5497,32.73,54,100,3.10,VE,1698421992
1,1,olyokminsk,60.3736,120.4264,2.60,89,100,2.33,RU,1698421992
2,2,karratha,-20.7377,116.8463,27.74,69,13,2.90,AU,1698421993
3,3,iskateley,67.6803,53.1512,-11.66,100,36,1.79,RU,1698421994
4,4,figuig (centre),32.1089,-1.2286,27.46,16,69,3.97,MA,1698421995


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size = ("Humidity"),
    scale = 0.80,
    color = "City"
)


# Display the map
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

df1 = city_data_df[city_data_df["Max Temp"].between(21,27)]
df2 = df1[df1["Wind Speed"]<4]
df3 = df2[df2["Cloudiness"] <=10]


# Drop any rows with null values


# Display sample data
df3


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,tacna,-18.0056,-70.2483,21.11,53,0,2.57,PE,1698421999
21,21,wailua homesteads,22.0669,-159.3780,21.24,73,0,3.60,US,1698422005
25,25,kapa'a,22.0752,-159.3190,21.99,73,0,3.60,US,1698421909
50,50,cabo san lucas,22.8909,-109.9124,26.95,52,1,0.89,MX,1698421706
74,74,santa maria del oro,25.9333,-105.3667,21.42,26,0,0.57,MX,1698422037
152,152,saint-pierre,-21.3393,55.4781,24.82,78,0,2.57,RE,1698422085
233,233,broome,42.2506,-75.8330,24.14,63,0,3.60,US,1698422136
260,260,bandar lampung,-5.4254,105.2580,24.14,92,8,1.39,ID,1698422151
268,268,at taj,24.2049,23.2857,26.27,22,1,1.95,LY,1698422155
286,286,protaras,35.0125,34.0583,26.66,85,2,1.66,CY,1698422166


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df3[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,tacna,PE,-18.0056,-70.2483,53,
21,wailua homesteads,US,22.0669,-159.3780,73,
25,kapa'a,US,22.0752,-159.3190,73,
50,cabo san lucas,MX,22.8909,-109.9124,52,
74,santa maria del oro,MX,25.9333,-105.3667,26,
152,saint-pierre,RE,-21.3393,55.4781,78,
233,broome,US,42.2506,-75.8330,63,
260,bandar lampung,ID,-5.4254,105.2580,92,
268,at taj,LY,24.2049,23.2857,22,
286,protaras,CY,35.0125,34.0583,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 1
params = {
    "categories": categories,
    "apiKey":geoapify_key,
    "limit": limit,   
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tacna - nearest hotel: Hotel La Torre
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
kapa'a - nearest hotel: Pono Kai Resort
cabo san lucas - nearest hotel: Comfort Rooms
santa maria del oro - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
broome - nearest hotel: No hotel found
bandar lampung - nearest hotel: Grand Anugerah
at taj - nearest hotel: No hotel found
protaras - nearest hotel: Capo Bay
tinogasta - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
al burayqah - nearest hotel: فندق ماس ليبيا
lihue - nearest hotel: Kauai Palms
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
tlahualilo de zaragoza - nearest hotel: No hotel found
parral - nearest hotel: Hotel Adriana
bela - nearest hotel: Hotel City Palace
kailua-kona - nearest hotel: Kona Seaside Hotel
makaha - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn

,City,Country,Lat,Lng,Humidity,Hotel Name
11,tacna,PE,-18.0056,-70.2483,53,Hotel La Torre
21,wailua homesteads,US,22.0669,-159.3780,73,Hilton Garden Inn Kauai Wailua Bay
25,kapa'a,US,22.0752,-159.3190,73,Pono Kai Resort
50,cabo san lucas,MX,22.8909,-109.9124,52,Comfort Rooms
74,santa maria del oro,MX,25.9333,-105.3667,26,No hotel found
152,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
233,broome,US,42.2506,-75.8330,63,No hotel found
260,bandar lampung,ID,-5.4254,105.2580,92,Grand Anugerah
268,at taj,LY,24.2049,23.2857,22,No hotel found
286,protaras,CY,35.0125,34.0583,85,Capo Bay


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size = ("Humidity"),
    scale = 0.80,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)


# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)